<a href="https://colab.research.google.com/github/ffedox/pbr/blob/main/text_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
import os
import re

In [28]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# This section is to merge .xlsx files

In [30]:
path = os.getcwd()
files = os.listdir(path)
files

['.config',
 'parallel_corpus_vg_en_it_3.xlsx',
 'drive',
 'parallel_corpus_vg_en_it_5.xlsx',
 'parallel_corpus_vg_en_it_19.xlsx',
 'parallel_corpus_vg_en_it_12.xlsx',
 'parallel_corpus_vg_en_it_13.xlsx',
 'parallel_corpus_vg_en_it_18.xlsx',
 'parallel_corpus_vg_en_it_10.xlsx',
 'parallel_corpus_vg_en_it_16.xlsx',
 'parallel_corpus_vg_en_it_15.xlsx',
 'parallel_corpus_vg_en_it_8.xlsx',
 'parallel_corpus_vg_en_it_20.xlsx',
 'parallel_corpus_vg_en_it_9.xlsx',
 'parallel_corpus_vg_en_it_14.xlsx',
 'parallel_corpus_vg_en_it_2.xlsx',
 'parallel_corpus_vg_en_it_11.xlsx',
 'parallel_corpus_vg_en_it_1.xlsx',
 'parallel_corpus_vg_en_it_7.xlsx',
 'parallel_corpus_vg_en_it_21.xlsx',
 'parallel_corpus_vg_en_it_4.xlsx',
 'parallel_corpus_vg_en_it_17.xlsx',
 'parallel_corpus_vg_en_it_6.xlsx',
 'sample_data']

In [31]:
files_xlsx = [f for f in files if f[-4:] == 'xlsx']
files_xlsx

['parallel_corpus_vg_en_it_3.xlsx',
 'parallel_corpus_vg_en_it_5.xlsx',
 'parallel_corpus_vg_en_it_19.xlsx',
 'parallel_corpus_vg_en_it_12.xlsx',
 'parallel_corpus_vg_en_it_13.xlsx',
 'parallel_corpus_vg_en_it_18.xlsx',
 'parallel_corpus_vg_en_it_10.xlsx',
 'parallel_corpus_vg_en_it_16.xlsx',
 'parallel_corpus_vg_en_it_15.xlsx',
 'parallel_corpus_vg_en_it_8.xlsx',
 'parallel_corpus_vg_en_it_20.xlsx',
 'parallel_corpus_vg_en_it_9.xlsx',
 'parallel_corpus_vg_en_it_14.xlsx',
 'parallel_corpus_vg_en_it_2.xlsx',
 'parallel_corpus_vg_en_it_11.xlsx',
 'parallel_corpus_vg_en_it_1.xlsx',
 'parallel_corpus_vg_en_it_7.xlsx',
 'parallel_corpus_vg_en_it_21.xlsx',
 'parallel_corpus_vg_en_it_4.xlsx',
 'parallel_corpus_vg_en_it_17.xlsx',
 'parallel_corpus_vg_en_it_6.xlsx']

In [32]:
df = pd.DataFrame()

In [33]:
for f in files_xlsx:
    data = pd.read_excel(f, index_col=0)
    df = df.append(data)

In [34]:
df = df.drop_duplicates()

In [35]:
df['en'] = df['en'].apply(lambda x: re.sub('\n', ' ', str(x))) # Replacing newlines with whitespaces
df['it'] = df['it'].apply(lambda x: re.sub('\n', ' ', str(x)))

In [36]:
df['en'] = df['en'].apply(lambda x: re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ\/\-.,;:"\']+', '', str(x))) # Remove special characters but not accented letters
df['it'] = df['it'].apply(lambda x: re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ\/\-.,;:"\']+', '', str(x)))

In [37]:
# Tokenizing the texts
df['tokenized_text_en'] = df['en'].apply(word_tokenize) 
df['token_count_en'] = df['tokenized_text_en'].apply(lambda x: len(x))

df['tokenized_text_it'] = df['it'].apply(word_tokenize) 
df['token_count_it'] = df['tokenized_text_it'].apply(lambda x: len(x))

In [ ]:
# Removing sentences shorter or equal to six tokens
df = df[(df['token_count_en'] >= 6)]

In [ ]:
# Removing segments with a token difference larger than 10
df['difference'] = df["token_count_en"] - df["token_count_it"]
df['difference'] = df['difference'].abs()
df2 = df[(df['difference'] > 10)] # Storing these so that we can analyze them later
df = df[(df['difference'] < 10)]

In [38]:
df['difference'] = df["token_count_en"] - df["token_count_it"]
df['difference'] = df['difference'].abs()

In [ ]:
total_tokens_en = df['token_count_en'].sum()
total_tokens_it = df['token_count_it'].sum()

In [71]:
mean_sentence_en = df['token_count_en'].mean()
mean_sentence_it = df['token_count_it'].mean()

In [72]:
print(mean_sentence_en)

26.727158529436394


In [73]:
print(mean_sentence_it)

27.742483879072104


In [ ]:
print(total_tokens_en)

259412


In [ ]:
print(total_tokens_it)

265407


In [ ]:
df = df.reset_index(drop=True)

In [ ]:
# df.to_excel("parallel_corpus.xlsx")  